In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

train = pd.read_csv('train.csv', parse_dates = ['date']) #parse dates = bikin format tanggal tanpa garing
train

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [2]:
test = pd.read_csv('test.csv', parse_dates = ['date'])
test

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50


In [3]:
print('Min date from train set : ' + str(train['date'].min().date()))
print('Max date from train set : ' + str(train['date'].max().date()))

Min date from train set : 2013-01-01
Max date from train set : 2017-12-31


In [4]:
lag_size = (test['date'].max().date() - train['date'].max().date()).days #perbedaan jarak waktu
print('Max date from train set : ' + str(train['date'].max().date()))
print('Max date from test set : ' + str(test['date'].max().date()))
print('Forecast lag size', lag_size)

Max date from train set : 2017-12-31
Max date from test set : 2018-03-31
Forecast lag size 90


In [5]:
dates = train.groupby('date', as_index = False)['sales'].sum()
stores = train.groupby(['store', 'date'], as_index = False)['sales'].sum()
items = train.groupby(['item', 'date'], as_index = False)['sales'].sum()
dates
stores
items

,item,date,sales
0,1,2013-01-01,133
1,1,2013-01-02,99
2,1,2013-01-03,127
3,1,2013-01-04,145
4,1,2013-01-05,149
...,...,...,...
91295,50,2017-12-27,511
91296,50,2017-12-28,587
91297,50,2017-12-29,596
91298,50,2017-12-30,612


In [9]:

import plotly.graph_objs as go
from plotly.offline import iplot

plot_dates = go.Scatter(x = dates['date'], y = dates['sales'])
layout = go.Layout(title = 'Daily sales', xaxis = dict(title = 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = [plot_dates], layout = layout)
# iplot(fig)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'type': 'scatter',
              'x': array([datetime.datetime(2013, 1, 1, 0, 0),
                          datetime.datetime(2013, 1, 2, 0, 0),
                          datetime.datetime(2013, 1, 3, 0, 0), ...,
                          datetime.datetime(2017, 12, 29, 0, 0),
                          datetime.datetime(2017, 12, 30, 0, 0),
                          datetime.datetime(2017, 12, 31, 0, 0)], dtype=object),
              'y': array([13696, 13678, 14488, ..., 23535, 24988, 26420], dtype=int64)}],
    'layout': {'template': '...',
               'title': {'text': 'Daily sales'},
               'xaxis': {'title': {'text': 'Date'}},
               'yaxis': {'title': {'text': 'Sales'}}}
})

In [ ]:
plot_stores = []
for store in stores['store'].unique():
    c_store = stores[(stores['store'] == store)]
    plot_stores.append(go.Scatter(x = c_store['date'], y = c_store['sales'], name = ('Store' + str(store))))
layout = go.Layout(title = 'Store daily sales', xaxis = dict(title = 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = plot_stores, layout = layout)
iplot(fig)

In [ ]:
plot_items = []
for item in items['item'].unique():
    c_items = items[(items['item'] == item)]
    plot_items.append(go.Scatter(x = c_items['date'], y = c_items['sales'], name = ('Item ' + str(item))))
layout = go.Layout(title = 'Item daily sales', xaxis = dict(title = 'Date'), yaxis = dict(title = 'Sales'))
fig = go.Figure(data = plot_items, layout = layout)
iplot(fig)

In [ ]:
indexes_train = []
for i in range(train.store.nunique()*train.item.nunique()):
    indexes_train = np.append(indexes_train, np.linspace(0, train.date.nunique()-1, train.date.nunique(), dtype = int))
    i=+1
indexes_train

In [ ]:
train["indexes"] = indexes_train
train

In [ ]:
data = pd.concat([train, test], sort = False)
data

In [ ]:
hour = 60*60
day = 24*hour
week = 7*day
year = 365.2425*day
df1 = data.copy()
df1["index_second"] = df1["indexes"]*day
df1


In [ ]:
df1["YEAR_index_norm"] = 2 * np.pi * df1["index_second"] / year
df1["YEAR_cos_index"] = np.cos(df1["YEAR_index_norm"])
df1["YEAR_sin_index"] = np.sin(df1["YEAR_index_norm"])
df1

In [ ]:
df1.iloc[0:740, 9].plot()

In [ ]:
df2 = df1.drop(["indexes", "index_second", "YEAR_index_norm"], axis = 1)
df2.head()

#https://docs.google.com/spreadsheets/d/1cN0yTFt-G6uxLWMsn9guc2WMuRIS38lpjK1luDVi-s0/edit#gid=620536088

In [ ]:
def create_date_time_features(df):
    df = df.copy()
    df['dayofweek'] = df.date.dt.dayofweek
    df['quarter'] = df.date.dt.quarter
    df['month'] = df.date.dt.month
    df['year'] = df.date.dt.year
    df['dayofyear'] = df.date.dt.dayofyear
    df['dayofmonth'] = df.date.dt.day
    df['weekofyear'] = df.date.dt.isocalendar().week.astype("int64")
    df["season"] = np.where(df.month.isin([12, 1, 2]),0, 1)
    df["season"] = np.where(df.month.isin([6, 7, 8]), 2, df['season'])
    df["season"] = np.where(df.month.isin([9, 10, 11]), 3, df["season"])
    return df

In [ ]:
df3 = df2.copy()
df3 = create_date_time_features(df3)
df3

In [ ]:
def lag_features(df, lags):
    for lag in lags:
        df['sales_lag_' + str(lag)] = df.groupby(["store", "item"])['sales'].transform(lambda x : x.shift(lag))
        df.fillna(0, inplace = True)

    return df

In [ ]:
df4 =  df3.copy()
lag_features(df4, [91, 98, 105, 112, 119, 126, 182, 364, 546, 728])
df4

In [ ]:
def roll_mean_features(df, windows):
    for window in windows:
        df['sales_roll_mean_' + str(window)] = df.groupby(["store", "item"])['sales'].transform(lambda x:x.shift(1).rolling(window = window, min_periods = 10, win_type = "triang").mean())
        df.fillna(0, inplace = True)
    return df

In [ ]:
df5 = df4.copy()
df5  = roll_mean_features(df5, [365, 546])
df5.tail(10)

In [ ]:
def ewm_features(dataframe, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            dataframe['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = dataframe.groupby(["store", "item"])['sales'].transform(lambda x: x.shift(lag).ewm(alpha = alpha).mean())
    dataframe.fillna(0, inplace = True)
    return dataframe

In [ ]:
df6 = df5.copy()
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [91, 98, 105, 112, 180, 270, 365, 546, 728]
df6 = ewm_features(df6, alphas, lags)
df6.tail()